In [ ]:
#————————————————————

# Name: Azure OpenAI Fine Tuning (V1)

# Purpose:


# Company: Allgeier Schweiz AG
# Author: Nicolas Rehder (nrehder@allgeier.ch)
# Create for: SDSC 2024
# Date Created: 22.01.2024
# Last Updated: 22.01.2024
# Python Version: 3.10.4

# General Sources:

# Azure Authentication Token
# https://learn.microsoft.com/en-us/cli/azure/account?view=azure-cli-latest#az-account-get-access-token()

# Azure Openai Usage:
# https://learn.microsoft.com/en-us/azure/ai-services/openai/tutorials/fine-tune?tabs=python-new%2Ccommand-line
# https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/fine-tuning?tabs=turbo%2Cpython&pivots=programming-language-python
# https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/chatgpt?tabs=python&pivots=programming-language-chat-completions#few-shot-learning-with-chat-completion

# Additionals:
# https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models

# Download Python packages (run the below command in terminal if packages have not yet been installed)
# pip install -r C:\Python\sdsc\requirements.txt

#————————————————————

In [2]:
# Import required libraries
import os
import io
import time
from io import StringIO
import json
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd
from openai import AzureOpenAI
import json
from IPython.display import clear_output
from IPython.core.display import HTML
import requests
import random

In [3]:
# Load required variables from env file.
load_dotenv(dotenv_path=Path("C:\Python\openai-lab\.venv\.env"))

# Load Azure OpenAI Key and Endpoint. These values can be found within the Azure OpenAI Service resource in portal.azure.com under Keys and Endpoint
azure_oai_key = os.environ['AZURE_OPENAI_KEY']
azure_oai_endpoint = os.environ['AZURE_OPENAI_ENDPOINT']

In [4]:
# Initialize the Azure OpenAI client
client = AzureOpenAI(
    api_key = azure_oai_key,  
    api_version = "2024-02-15-preview",
    azure_endpoint = azure_oai_endpoint
    )

In [5]:
recipe = "Ah, great ingredients! With avocado, coconut milk, tofu, and soy sauce, I have the perfect Mediterranean-Asian fusion recipe for you – Coconut Avocado Tofu Curry with Soy Sauce Glazed Tofu.\n\nLet's get started!\n\nCoconut Avocado Tofu Curry:\nIngredients:\n- 1 tablespoon cooking oil\n- 1 red onion, finely diced\n- 3 garlic cloves, minced\n- 1 tablespoon fresh ginger, grated\n- 1 tablespoon red curry paste (adjust to taste)\n- 1 can (400ml) coconut milk\n- 1 cup vegetable broth (adjust for desired consistency)\n- 1 tablespoon soy sauce\n- 2 cups of mixed vegetables (such as bell peppers, snap peas, or zucchini), diced\n- 1 cup cherry tomatoes, halved\n- Avocado, sliced, for serving\n- Fresh cilantro, for serving\n- Lime wedges, for serving\n- Rice or noodles, for serving\n\nInstructions:\n1. In a large pan, heat the cooking oil over medium heat and add the diced red onion, minced garlic, and grated ginger. Sauté until fragrant and the onions are translucent.\n2. Add the red curry paste and stir it in with the onions and aromatics. Let it cook for a minute or two, until fragrant.\n3. Pour in the coconut milk, vegetable broth, and soy sauce. Bring it to a gentle simmer.\n4. Add the mixed vegetables and cherry tomatoes. Allow them to cook until tender but still slightly crunchy.\n5. While the curry is simmering, prepare the soy sauce-glazed tofu.\n\nSoy Sauce Glazed Tofu:\nIngredients:\n- 1 pack of firm tofu (about 14 oz), cut into cubes\n- 2 tablespoons soy sauce\n- 2 tablespoons maple syrup\n- 1 teaspoon cornstarch\n\nInstructions:\n1. In a separate pan, heat a bit of cooking oil over medium heat. Add the tofu cubes and cook until golden brown on all sides.\n2. In a small bowl, whisk together the soy sauce, maple syrup, and cornstarch until smooth.\n3. Pour the sauce over the pan with tofu and stir to coat the tofu cubes evenly. Let it cook for a few minutes until the sauce thickens and glazes over the tofu.\n\nServing:\n1. Serve the Coconut Avocado Tofu Curry over rice or noodles.\n2. Garnish with sliced avocado, fresh cilantro, and a squeeze of lime juice.\n3. Serve alongside the soy sauce-glazed tofu cubes.\n\nEnjoy this Mediterranean-Asian fusion feast!"

In [6]:
entity_extraction = client.chat.completions.create(
    model="gpt-35-turbo", # model = "deployment_name".
    messages=[{"role":"system","content":"You are an assistant designed to extract entities from text. Users will paste in a string of text and you will respond with entities you've extracted from the text as a JSON object. Here's an example of your output format:\n{\n   \"recipe name\": \"\",\n   \"ingredients\": \"\",\n   \"amount\": \"\"\n}"},
                {"role":"user","content":"Ingredients:\n- 1 cucumber\n- 1 capsicum (any color)\n- 1 cup flour (or gluten-free flour)\n- 2 tablespoons soy sauce\n- Oil for frying\n\nInstructions:\n1. Prepare the vegetables:\n   - Slice the cucumber and capsicum into thin, even strips.\n   - Set them aside.\n\n2. Make the batter:\n   - In a mixing bowl, combine the flour and soy sauce.\n   - Gradually add water until a thick, smooth batter forms.\n   - Dip the cucumber and capsicum strips into the batter, ensuring they are evenly coated.\n\n3. Fry the vegetables:\n   - Heat oil in a deep pan or skillet.\n   - Fry the battered cucumber and capsicum strips until they turn golden brown and crispy.\n   - Remove from oil and drain on a paper towel to remove excess oil.\n\n4. Serve:\n   - Arrange the fried cucumber and capsicum strips on a serving platter.\n   - Sprinkle with a little sea salt for extra flavor.\n   - Serve with a side of soy sauce or your favorite dipping sauce.\n\nEnjoy this Asian-inspired vegan appetizer of crispy battered cucumber and capsicum that combines texture and flavor for a heavenly first course!"},
                {"role":"assistant","content":"{\n   \"recipe name\": \"Asian-inspired vegan appetizer\",\n   \"ingredients\": \"cucumber, capsicum, flour, soy sauce, oil, water, sea salt\",\n   \"amount\": \"1 unit, 1 cup, 2 tablespoons, to taste, to taste, to taste\"\n}"},
                {"role": "user", "content": recipe}
				]
)

In [10]:
recipe_extraction = entity_extraction.choices[0].message.content

In [ ]:
with open('recipe_extraction.jsonl', 'w', encoding='utf-8') as f:
    json.dump(recipe_extraction, f)

In [13]:
import pprint
pprint.pprint(recipe_extraction)

('{\n'
 '   "recipe name": "Coconut Avocado Tofu Curry with Soy Sauce Glazed Tofu",\n'
 '   "ingredients": "cooking oil, red onion, garlic cloves, fresh ginger, red '
 'curry paste, coconut milk, vegetable broth, soy sauce, mixed vegetables, '
 'cherry tomatoes, avocado, fresh cilantro, lime wedges, rice or noodles, firm '
 'tofu, maple syrup, cornstarch",\n'
 '   "amount": "1 tablespoon, 1 unit, 3 units, 1 tablespoon, 1 can (400ml), 1 '
 'cup, 1 tablespoon, 2 cups, 1 cup, to taste, to taste, to taste, to taste, 14 '
 'oz, 2 tablespoons, 2 tablespoons, 1 teaspoon"\n'
 '}')


In [8]:
text = entity_extraction.choices[0].message.content
text_clean = text.replace("\n", "")
print (text_clean)

{   "recipe name": "Coconut Avocado Tofu Curry with Soy Sauce Glazed Tofu",   "ingredients": "cooking oil, red onion, garlic cloves, fresh ginger, red curry paste, coconut milk, vegetable broth, soy sauce, mixed vegetables, cherry tomatoes, avocado, fresh cilantro, lime wedges, rice or noodles, firm tofu, maple syrup, cornstarch",   "amount": "1 tablespoon, 1 unit, 3 units, 1 tablespoon, 1 can (400ml), 1 cup, 1 tablespoon, 2 cups, 1 cup, to taste, to taste, to taste, to taste, 14 oz, 2 tablespoons, 2 tablespoons, 1 teaspoon"}
